# Summary
Merges the the StatCan dissemination areas and Ookla tile overlay pre-pickled data. 
Calculates aggregates from the ookla tile data over the most recent 4 quarters and addes the averages 
to the overlay tiles. 


In [1]:
import pickle 
import src.config
import pandas as pd

from src.datasets.loading import statcan, ookla

output_dir = src.config.DATA_DIRECTORY / 'processed' 
output_dir.mkdir(exist_ok = True)
feature_pickle = output_dir / 'feature_table.pickle'
all_tiles_pickle = output_dir / 'full_tileset.pickle'


In [2]:
with open(all_tiles_pickle,'rb') as f:
    full_data = pickle.load(f)

In [3]:
full_data.columns

Index(['DAUID', 'PRUID', 'PRNAME', 'CDUID', 'CDNAME', 'CDTYPE', 'CCSUID',
       'CCSNAME', 'CSDUID', 'CSDNAME', 'CSDTYPE', 'ERUID', 'ERNAME', 'SACCODE',
       'SACTYPE', 'CMAUID', 'CMAPUID', 'CMANAME', 'CMATYPE', 'CTUID', 'CTNAME',
       'ADAUID', 'das_area', 'quadkey', 'tile_area', 'tile_frac', 'das_frac',
       'geometry', 'index_right', 'PCUID', 'PCNAME', 'PCTYPE', 'PCPUID',
       'PCCLASS', 'GEO_NAME', 'DAPOP'],
      dtype='object')

In [4]:
to_merge = full_data.dropna(subset=['DAUID','quadkey']).sort_values(by=['quadkey','tile_frac'],ascending=False).drop_duplicates(subset='quadkey', keep='first')

In [5]:
speed_data = ookla.speed_data()
year_quarters = sorted(speed_data.loc[:,['year','quarter']].value_counts().index.values)
last4 = year_quarters[-4:]
last_4Q_speed = speed_data[speed_data.loc[:,['year','quarter']].apply(tuple, axis=1).isin(last4)]

last4Q_agg = last_4Q_speed.groupby(['quadkey','conn_type']).agg({
    'avg_d_kbps':'mean',
    'avg_u_kbps':'mean',
    'tests':'sum',
    'devices':'sum',
    'avg_lat_ms':'mean'
}).reset_index()
last4Q_agg['quadkey'] = last4Q_agg['quadkey'].astype(str)
last4Q_agg

,quadkey,conn_type,avg_d_kbps,avg_u_kbps,tests,devices,avg_lat_ms
0,13230212122122,fixed,544197.00,269445.0,1,1,19.00
1,23313133202223,fixed,43221.00,3756.0,2,1,134.00
2,23313133220001,fixed,29290.00,2404.0,2,1,255.00
3,23313203322102,fixed,76875.00,21079.0,2,1,76.00
4,23331133131233,fixed,13775.00,525.0,2,1,28.00
...,...,...,...,...,...,...,...
426107,303303002122001,fixed,31970.75,10417.5,7,4,58.25
426108,303303002122002,fixed,5414.50,2030.0,2,2,59.00
426109,303303002122003,fixed,37499.00,436.0,1,1,48.00
426110,303303002331120,fixed,63916.00,16396.0,1,1,60.00


In [6]:
feature_table = to_merge.merge(last4Q_agg.loc[lambda s:s.conn_type=='fixed'], on='quadkey')

In [7]:
feature_table.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 292548 entries, 0 to 292547
Data columns (total 42 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   DAUID        292548 non-null  object  
 1   PRUID        284769 non-null  object  
 2   PRNAME       284769 non-null  object  
 3   CDUID        284769 non-null  object  
 4   CDNAME       284769 non-null  object  
 5   CDTYPE       284769 non-null  object  
 6   CCSUID       284769 non-null  object  
 7   CCSNAME      284769 non-null  object  
 8   CSDUID       284769 non-null  object  
 9   CSDNAME      284769 non-null  object  
 10  CSDTYPE      284769 non-null  object  
 11  ERUID        284769 non-null  object  
 12  ERNAME       284769 non-null  object  
 13  SACCODE      284769 non-null  object  
 14  SACTYPE      284769 non-null  object  
 15  CMAUID       146048 non-null  object  
 16  CMAPUID      146048 non-null  object  
 17  CMANAME      146048 non-null  object  
 

In [8]:
with open(feature_pickle, 'wb') as f:
    pickle.dump(feature_table, f)